In [1]:
#%%

from music21 import *
import copy, random, math

def beat_strength(offset: float) -> int:
    # Returns 2 if on 1, 3
    # 1 if on 2, 4
    # 0 if on +
    beat = math.floor(offset)
    decimal = offset - beat
    if decimal == 0:
        return 2 if beat in [0, 2] else 1
    else:
        return 0
    
def next_chord_tone(p: pitch.Pitch, ch: chord.Chord) -> pitch.Pitch:
    next_map = {
        ch.root().pitchClass: ch.third,
        ch.third.pitchClass: ch.fifth,
        ch.fifth.pitchClass: ch.root(),
    }
    return next_map[p.pitchClass].transposeAboveTarget(p)

def prev_chord_tone(p: pitch.Pitch, ch: chord.Chord) -> pitch.Pitch:
    prev_map = {
        ch.root().pitchClass: ch.fifth,
        ch.third.pitchClass: ch.root(),
        ch.fifth.pitchClass: ch.third,
    }
    return prev_map[p.pitchClass].transposeBelowTarget(p)

def chord_tones_above(p: pitch.Pitch, ch: chord.Chord, k: int) -> list[pitch.Pitch]:
    '''
    Return the k chord tones above and including p.
    '''
    res = [p]
    for _ in range(k - 1):
        res.append(next_chord_tone(res[-1], ch))
    return res

class BassModifier:
    def __init__(self, score_path: str):
        '''
        Takes as input a path to a score which contains the chord labels
        '''
        self.sc = converter.parse(score_path)
        self.bassline = self.sc.parts[1]
        self.scale = self.bassline.keySignature.getScale('major')
        self.chords = {}
        self.notes = {}
        for n in self.bassline.flatten()[note.Note]:
            self.notes[n.offset] = n
            p = n.pitch
            self.chords[n.offset] = chord.Chord(
                [p, self.scale.nextPitch(p, stepSize=2), self.scale.nextPitch(p, stepSize=4)],
                duration=n.duration
            )
        self.measures = self.bassline[stream.Measure]
    
    def set_rhythm(self, n: note.Note, pattern: list[int]):
        '''
        Modifies a notes rhythm (where it is in a measure's context)
        '''
        d = n.quarterLength
        m = self.bassline.measure(n.measureNumber)
        curr_offset = n.offset # - m.offset
        # print("Starting at", offset, m.offset, n.measureNumber)
        denom = sum(pattern)
        for ix, amt in enumerate(pattern):
            new_d = amt / denom * d
            # print("Inserting", new_d, "at", curr_offset)
            if ix == 0:
                n.quarterLength = new_d
            else:
                new_n = note.Note(n.pitch, duration=duration.Duration(new_d))
                m.insert(curr_offset, new_n)
            curr_offset += new_d

    def inject_rhythm(self):
        '''
        For a loop of n chords, choose two patterns, one for first n-1, and a different
        one for the nth (to give rhythmic drive to the next loop)
        
        Currently, breaks into half-note divisions and provides one of the following
        distributions: 
        3:3:2, 3:1, 2:2, 1:1:1:1, 2:1:1, 1:1:2
        
        Operate on each half note first. Then organize which ones are special

        Todo: make more decisions on the weighting.
        '''
        # n = len(self.bassline.measures())
        # First pattern for the first n-1 measures
        weights = [4, 3, 3, 1, 3, 2]
        choices = [
            [3, 3, 2],
            [3, 1],
            [1, 1],
            [1, 1, 1, 1],
            [2, 1, 1],
            [1, 1, 2]
        ]
        selected_patterns = random.choices(choices, weights=weights, k=3)
        
        for m in self.measures[:-1]:
            for n in m[note.Note]:
                # Divide it by 2
                self.set_rhythm(n, [2, 1, 1])
            for n in m[note.Note]:
                self.set_rhythm(n, selected_patterns[0])

        for n in self.measures[-1][note.Note]:
            self.set_rhythm(n, selected_patterns[2])

    def arpeggiate(self):
        '''
        Set more diverse pitches for the rhythms
        '''
        # Randomly shift by amounts. 
        weights = [4, 3, 2, 3, 1]
        choices = [
            0, 1, 2, 3, 4 # Root, 3rd, 5th, 8th, 10th for now.
        ]
        selected_pattern = [0] + random.choices(choices, weights=weights, k=3)
        curr_chord = self.chords[0]
        chord_tones = chord_tones_above(curr_chord.root(), curr_chord, len(choices))
        pattern_pos = 0
        for m in self.measures:
            for n in m[note.Note]:
                # TODO: Maybe consider strong vs weak beats?
                # Cycle through the pattern for the current chord
                n.pitch = chord_tones[selected_pattern[pattern_pos]]
                pattern_pos = (pattern_pos + 1) % len(selected_pattern)

                curr_offset = m.offset + n.offset 
                if curr_offset in self.chords:
                    curr_chord = self.chords[curr_offset]
                    chord_tones = chord_tones_above(curr_chord.root(), curr_chord, len(choices))
                    pattern_pos = 0
    
    
    def thicken(self):
        '''
        Expand the notes to contain more chords, upwards
        '''
    
    def insert_approach_tones(self):
        '''
        Override some of the previous rhythms with approach tones in front of new chords
        '''
        # for offset, ch

    '''
    First pass: Adding rhythm 
    Split into half notes. Then, choose patterns for each quarter
    
    Choose two patterns and place one for the final transition to give energy
    
    Second pass: Adding pitch 
    Choosing the arpeggios
    
    Third pass: Extending chords upward
    '''
    
    

#%%

score_dir = '/Users/derrick/PycharmProjects/MIT/21M_383/music-melody-generator/scores/'
bm = BassModifier(score_dir + "Touch.mxl")

# list(bv.bassline.flatten()[note.Note].getElementsByOffset(4))[0].duration

bm.inject_rhythm()
bm.bassline.show()
bm.arpeggiate()
bm.bassline.show()

#%%

In [1]:
# bm.bassline.write('midi', 'output.midi')
print("Hello")

In [ ]:
def play_midi():
    file_path = 'output.midi'
    bm.bassline.write('midi', file_path)

    mf = midi.MidiFile()
    mf.open(file_path)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    sp = midi.realtime.StreamPlayer(s)
    sp.play()

In [ ]:
play_midi('output.midi')

StreamPlayerException: StreamPlayer requires pygame.  Install first